# Section 2: Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and upload _model updates_ to a central server.

Use Cases:

- app company (Texting prediction app)
- predictive maintenance (automobiles / industrial engines)
- wearable medical devices
- ad blockers / autotomplete in browsers (Firefox/Brave)

Challenge Description, data is distributed among sources but we cannot aggregated it because of:

- privacy concerns: legal, user discomfort, competitive dynamics
- engineering: the bandwidth/storage requirements of aggregating the larger dataset
    
## Lesson1: Basic Remote Execution in PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. We need the ability to tell remote machines to execute the operations required for Deep Learning. Instead of using Torch tensors, we're now going to work with **pointers** to tensors.

In [1]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)

# NOTE: we are using pytorch, the warning related to TensorFlow is irrelevant.

In [43]:
'''
In this project we will demostrate how to send data to a virtual remote worker.
First, let's create a "remote" machine owned by a "remote" person. We'll call him Bob.
'''

bob = sy.VirtualWorker(hook, id="bob")
print("--------Before send--------")
print("bob._objects:", bob._objects)
print("")

x = th.tensor([1,2,3,4,5])
x = x.send(bob)

print("--------After send--------")
print("bob._objects", bob._objects)
print("x", x)
print("x.location", x.location)
print("x.id_at_location", x.id_at_location)
print("x,id", x.id)
print("x.owner", x.owner)
print("hook.local_worker", hook.local_worker)
print("")

x = x.get()

print("--------After Get--------")
print("bob._objects", bob._objects)
print("")

--------Before send--------
bob._objects: {31309081243: tensor([1, 2, 3, 4, 5])}

--------After send--------
bob._objects {31309081243: tensor([1, 2, 3, 4, 5]), 87097560991: tensor([1, 2, 3, 4, 5])}
x (Wrapper)>[PointerTensor | me:94431760384 -> bob:87097560991]
x.location <VirtualWorker id:bob #objects:2>
x.id_at_location 87097560991
x,id 94431760384
x.owner <VirtualWorker id:me #objects:0>
hook.local_worker <VirtualWorker id:me #objects:0>

--------After Get--------
bob._objects {31309081243: tensor([1, 2, 3, 4, 5])}



In [3]:
'''
In this project, 
I want you to .send() and .get() a tensor to two workers by calling .send(bob,alice). 
This will first require the creation of another VirtualWorker called alice.
'''

alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5])

x_ptr = x.send(bob, alice)
x_ptr.get()

x = th.tensor([1,2,3,4,5]).send(bob, alice)

x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [4]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

print("x", x)
print("y", y)

z = x + y
print("z", z)

z = z.get()
print("z", z)

z = th.add(x,y)
print("z", z)

z = z.get()
print("z", z)

x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

z = (x + y).sum()
z.backward()

x = x.get()

print("x", x)
print("x.grad", x.grad)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [44]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [21]:
bob = bob.clear_objects()

In [22]:
bob._objects

{}

In [23]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [24]:
bob._objects

{72397212561: tensor([1, 2, 3, 4, 5])}

In [25]:
del x

In [26]:
bob._objects

{}

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [28]:
bob._objects

{639091026: tensor([1, 2, 3, 4, 5])}

In [29]:
x = "asdf"

In [30]:
bob._objects

{}

In [31]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [32]:
x

(Wrapper)>[PointerTensor | me:87360472634 -> bob:60251605741]

In [33]:
bob._objects

{60251605741: tensor([1, 2, 3, 4, 5])}

In [34]:
x = "asdf"

In [35]:
bob._objects

{60251605741: tensor([1, 2, 3, 4, 5])}

In [36]:
del x

In [37]:
bob._objects

{60251605741: tensor([1, 2, 3, 4, 5])}

In [38]:
bob = bob.clear_objects()
bob._objects

{}

In [39]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [40]:
bob._objects

{36592179336: tensor([1, 2, 3, 4, 5])}

In [41]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [42]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:30717511465 -> bob:31309081243]
Tensor B: tensor([1, 1, 1, 1, 1])

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [ ]:
train()

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
alice._objects

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
alice._objects